In [ ]:
####### README:
#### THE Dn/DS written in the files generated here are actually the Pn/Ps normalized, Dn/Ds is computed between species (human-mouse) in the other scritps.


### Load Data
def reverse_complement(seq):
    dna_seq = seq.upper()
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in reversed(dna_seq))

## EE
ee_pn={}
ee_ps={}
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/hg38_ee_gnomAD_pNpS.tsv") as file:
    for line in file:   
        try:         
            ee_pn[line.strip().split()[0]] = int(line.strip().split()[2])
            ee_ps[line.strip().split()[0]] = int(line.strip().split()[3])
        except ValueError:
            continue

ee_seq_dna={}
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/ee_dna_seqs.tsv") as file:
    for line in file:            
        if line.strip().split()[0][-1] == "f":
            ee_seq_dna[line.strip().split()[0]] = line.strip().split()[1]
        else:
            ee_seq_dna[line.strip().split()[0]] = reverse_complement(line.strip().split()[1])

ee_seq_aa={}
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/ee_aa_seqs_hg38.txt") as file:
    for line in file:            
        ee_seq_aa[line.strip().split()[0]] = line.strip().split()[1]

## NEG
neg_pn={}
neg_ps={}
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/hg38_neg_gnomAD_pNpS.tsv") as file:
    for line in file:            
        try:
            neg_pn[line.strip().split()[0]] = int(line.strip().split()[2])
            neg_ps[line.strip().split()[0]] = int(line.strip().split()[3])
        except ValueError:
            continue

neg_seq_dna={}
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/ctrlNeg_dna_seqs.tsv") as file:
    for line in file:    
        if line.strip().split()[0][-1] == "f":        
            neg_seq_dna[line.strip().split()[0]] = line.strip().split()[1]
        else:
            neg_seq_dna[line.strip().split()[0]] = reverse_complement(line.strip().split()[1])

neg_seq_aa={}
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/neg_aa_seqs_hg38.txt") as file:
    for line in file:       
        try:    
            if line.strip().split()[1] != "false":
                neg_seq_aa[line.strip().split()[0]] = line.strip().split()[1]
        except IndexError:
            continue

In [4]:
### Functions
from Bio.Seq import Seq

def map_aa_to_dna(dna_seq, aa_seq):
    dna_seq = dna_seq.upper()
    aa_seq = aa_seq.upper()
    
    for offset in range(3):
        # translate from the current offset
        translated_seq = str(Seq(dna_seq[offset:]).translate(to_stop=False))
        
        # Case 1: Exact match starting from current offset
        if translated_seq.startswith(aa_seq):
            return offset, aa_seq, translated_seq
        
        # Case 2: Skip first AA because incomplete codon (missing bases)
        if translated_seq.startswith(aa_seq[1:]):
            return offset, aa_seq[1:], translated_seq
        
        # Case 3: translated_seq missing first AA, check starting from second AA
        if translated_seq[1:].startswith(aa_seq[1:]):
            return offset + 3, aa_seq[1:], translated_seq[1:]
        
    return None, None, None

def count_sites(cds_seq):
    """Count possible synonymous and nonsynonymous sites in CDS."""
    S_sites, N_sites = 0.0, 0.0
    cds_seq = cds_seq.upper()

    for i in range(0, len(cds_seq) - 2, 3):
        codon = cds_seq[i:i+3]
        if "N" in codon or len(codon) != 3:
            print("Test0")
            continue

        try:
            ref_aa = Seq(codon).translate()
        except:
            print("Test")
            continue

        syn, nonsyn = 0, 0
        for pos in range(3):
            for base in "ACGT":
                if base == codon[pos]:
                    continue
                mutant = list(codon)
                mutant[pos] = base
                mutant_codon = "".join(mutant)
                try:
                    mutant_aa = Seq(mutant_codon).translate()
                except:
                    continue
                if mutant_aa == ref_aa:
                    syn += 1
                else:
                    nonsyn += 1

        S_sites += syn / 9.0
        N_sites += nonsyn / 9.0

    return S_sites, N_sites

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Data import CodonTable
import csv

total_pN = total_pS = total_N_sites = total_S_sites = 0.0
total_dnds_ee = []
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/normalized_pnps_ee.tsv", "w", newline='') as out:
    writer = csv.writer(out, delimiter='\t')
    writer.writerow(["ExonID", "pN", "pS", "N_sites", "S_sites", "dN", "dS", "pN/pS"])

    for exon_id, dna_seq in ee_seq_dna.items():
        if exon_id in ee_seq_aa:
            aa_seq = ee_seq_aa[exon_id]
        else:
            continue

        offset, matched_aa, translated_dna = map_aa_to_dna(dna_seq, aa_seq)
        trimmed_dna = dna_seq[offset:]
        pN = ee_pn[exon_id]
        pS = ee_ps[exon_id]
        S_sites, N_sites = count_sites(trimmed_dna)

        total_pN += pN
        total_pS += pS
        total_N_sites += N_sites
        total_S_sites += S_sites

        dN = pN / N_sites if N_sites > 0 else 0
        dS = pS / S_sites if S_sites > 0 else 0
        if dS == 0:
            dn_ds = 0 if dN == 0 else "Inf"
        else:
            dn_ds = round(dN / dS, 6)

        total_dnds_ee.append(dn_ds)
        writer.writerow([exon_id, pN, pS, round(N_sites, 2), round(S_sites, 2),
                            round(dN, 6), round(dS, 6), dn_ds])
    
    # === Compute overall dN/dS
    dN_total = total_pN / total_N_sites if total_N_sites > 0 else 0
    dS_total = total_pS / total_S_sites if total_S_sites > 0 else 0
    if dS_total == 0:
        dN_dS_total = 0 if dN_total == 0 else "Inf"
    else:
        dN_dS_total = round(dN_total / dS_total, 6)
    writer.writerow(["Total", total_pN, total_pS, round(total_N_sites, 2), round(total_S_sites, 2),
                            round(dN_total, 6), round(dS_total, 6), dN_dS_total])


/home/mouren/miniconda3/lib/python3.9/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [ ]:
### NEG
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Data import CodonTable
import csv

total_pN = total_pS = total_N_sites = total_S_sites = 0.0
total_dnds_neg = []
with open("/mnt/project/exonhancer/ZENODO_REPO_DISCARDED_JC_DONT_TOUCH/gnomADv3_analysis/0_get_snp_gnomad/exons/vcf_by_chr/normalized_pnps_neg.tsv", "w", newline='') as out:
    writer = csv.writer(out, delimiter='\t')
    writer.writerow(["ExonID", "pN", "pS", "N_sites", "S_sites", "dN", "dS", "pN/pS"])

    for exon_id, dna_seq in neg_seq_dna.items():
        if exon_id in neg_seq_aa:
            aa_seq = neg_seq_aa[exon_id]
        else:
            continue

        offset, matched_aa, translated_dna = map_aa_to_dna(dna_seq, aa_seq)
        trimmed_dna = dna_seq[offset:]
        pN = neg_pn[exon_id]
        pS = neg_ps[exon_id]
        S_sites, N_sites = count_sites(trimmed_dna)

        total_pN += pN
        total_pS += pS
        total_N_sites += N_sites
        total_S_sites += S_sites

        dN = pN / N_sites if N_sites > 0 else 0
        dS = pS / S_sites if S_sites > 0 else 0
        if dS == 0:
            dn_ds = 0 if dN == 0 else "Inf"
        else:
            dn_ds = round(dN / dS, 6)

        total_dnds_neg.append(dn_ds)
        writer.writerow([exon_id, pN, pS, round(N_sites, 2), round(S_sites, 2),
                            round(dN, 6), round(dS, 6), dn_ds])
    
    # === Compute overall dN/dS
    dN_total = total_pN / total_N_sites if total_N_sites > 0 else 0
    dS_total = total_pS / total_S_sites if total_S_sites > 0 else 0
    if dS_total == 0:
        dN_dS_total = 0 if dN_total == 0 else "Inf"
    else:
        dN_dS_total = round(dN_total / dS_total, 6)
    writer.writerow(["Total", total_pN, total_pS, round(total_N_sites, 2), round(total_S_sites, 2),
                            round(dN_total, 6), round(dS_total, 6), dN_dS_total])


In [21]:
### STATS
from scipy.stats import chi2_contingency,mannwhitneyu
import numpy as np

# Convert to float, remove invalids
def clean_numeric(data):
    cleaned = []
    for v in data:
        try:
            f = float(v)
            #if not np.isnan(f):
            if np.isfinite(f):  # excludes NaN, inf, -inf
                cleaned.append(f)
        except (ValueError, TypeError):
            continue
    return cleaned

clean_ee = clean_numeric(total_dnds_ee)
clean_neg = clean_numeric(total_dnds_neg)

print("median", np.median(clean_ee))
print("median", np.median(clean_neg))

print("mean", np.mean(clean_ee))
print("mean", np.mean(clean_neg))

# Perform Mann-Whitney U test
stat, p_value = mannwhitneyu(clean_ee, clean_neg, alternative='two-sided')

print(f"\nMann-Whitney U Test:")
print(f"U statistic = {stat}")
print(f"p-value = {p_value}")

if p_value < 0.05:
    print("→ Significant difference in medians (p < 0.05)")
else:
    print("→ No significant difference in medians (p ≥ 0.05)")

median 0.586982
median 0.551181
mean 0.6326513612802613
mean 0.6497246409420001

Mann-Whitney U Test:
U statistic = 90424349.0
p-value = 1.0960299098202904e-21
→ Significant difference in medians (p < 0.05)


In [14]:
### total stats
# Build 2x2 table
table = np.array([
    [547885.0, 284408.0],        # Disease: [pN, pS]
    [149607.0, 68268.0]  # Control: [pN, pS]
])

chi2, p, dof, expected = chi2_contingency(table)

print(f"Chi-squared: {chi2:.4f}")
print(f"p-value: {p:.4g}")
print("Expected counts:\n", expected)

Chi-squared: 623.4202
p-value: 1.349e-137
Expected counts:
 [[552785.56302992 279507.43697008]
 [144706.43697008  73168.56302992]]
